# Quantum Chemistry Demo

In [8]:
# Install PySCF and rdkit
!pip install pyscf rdkit-pypi pandas

In [13]:
# Load Libraries
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, scf
import pandas as pd
import numpy as np

**PySCF:** Python-based Simulations of Chemistry Framework (PySCF) is an ab initio computational chemistry program natively implemented in Python program language.

**RDKit** is an open-source cheminformatics toolkit implemented in Python and C++. It provides robust tools for molecular representation, structure manipulation, and property calculation, enabling efficient handling of chemical information and integration into Python-based workflows.

**NumPy** is a fundamental package for scientific computing in Python. It provides efficient support for large, multi-dimensional arrays and matrices, along with a comprehensive collection of mathematical functions to perform element-wise operations, linear algebra, Fourier analysis, and more, forming the core of many numerical workflows in computational science.

**pandas** is a powerful, open-source data analysis and manipulation library for Python. It provides high-performance data structures like DataFrame and Series, enabling intuitive handling, transformation, and analysis of structured data within scientific and statistical workflows.

In [10]:
# Define molecules
molecules = {
    "water": "O",
    "ammonia": "N",
    "hydrogen_fluoride": "[H]F",
    "carbon_monoxide": "[C-]#[O+]",  # Charge-separated CO
    "methane": "C"
}

In [14]:
results = []

for name, smiles in molecules.items():
    # Generate 3D geometry
    mol_rdkit = Chem.AddHs(Chem.MolFromSmiles(smiles))
    AllChem.EmbedMolecule(mol_rdkit, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol_rdkit)
    conf = mol_rdkit.GetConformer()

    # Convert to PySCF format
    atoms = []
    for atom in mol_rdkit.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        atoms.append(f"{atom.GetSymbol()} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}")

    # PySCF Molecule
    mol = gto.Mole()
    mol.atom = "\n".join(atoms)
    mol.basis = "def2svp"
    mol.charge = 0
    mol.spin = 0
    mol.verbose = 0
    mol.build()

    # Hartree–Fock calculation
    mf = scf.RHF(mol)
    energy = mf.kernel()

    # Dipole moment
    dip = mf.dip_moment()  # returns (dx, dy, dz)
    dip_magnitude = np.linalg.norm(dip)

    # Store results
    results.append({
        "Molecule": name,
        "HF Energy (Hartree)": energy,
        "Dipole Moment (D)": f"({dip[0]:.3f}, {dip[1]:.3f}, {dip[2]:.3f})",
        "Dipole |μ| (D)": dip_magnitude
    })

# Display as DataFrame
df = pd.DataFrame(results)
df

Dipole moment(X, Y, Z, Debye): -0.00213, -2.16660, -0.00000
Dipole moment(X, Y, Z, Debye): -0.00482, -0.06263, -1.79608
Dipole moment(X, Y, Z, Debye): -2.12794,  0.00000, -0.00000
Dipole moment(X, Y, Z, Debye):  0.22638, -0.00000,  0.00000
Dipole moment(X, Y, Z, Debye): -0.00000,  0.00000, -0.00000


,Molecule,HF Energy (Hartree),Dipole Moment (D),Dipole |μ| (D)
0,water,-75.957213,"(-0.002, -2.167, -0.000)",2.166596e+00
1,ammonia,-56.145161,"(-0.005, -0.063, -1.796)",1.797180e+00
2,hydrogen_fluoride,-99.925116,"(-2.128, 0.000, -0.000)",2.127938e+00
3,carbon_monoxide,-112.645932,"(0.226, -0.000, 0.000)",2.263779e-01
4,methane,-40.168162,"(-0.000, 0.000, -0.000)",6.917209e-07
